# Import Dependencies

In [1]:
from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Connect to sqlite database
engine = create_engine('sqlite:////absolutepath/sqlite;', echo=False)
conn = engine.connect()

In [3]:
# Confirm connection by printing tables
engine.table_names()

['Country',
 'League',
 'Match',
 'Player',
 'Player_Attributes',
 'Team',
 'Team_Attributes',
 'sqlite_sequence']

# Examine Data

In [4]:
sql_view = "SELECT * FROM Match LIMIT 5;"

In [5]:
# Run query
view_data = pd.read_sql(sql_view, conn)
# Displaying data
view_data

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,None,None,None,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,None,None,None,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,None,None,None,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,None,None,None,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None

# Example 1: Simple Subsquery in <font color="red">FROM</font> Clause

<strong>Generic Synthax</strong><br>
SELECT column(s)<br>
FROM subquery<br>
WHERE conditions<br>
ORDER BY column(s)/alias(es) ASC/DESC<br>
LIMIT some_number;<br>

## Goal: Home goal average for every team

 - Step 1: Identify the top three teams that had the highest average home goals made in the 2011-2012 season

## Step 1: Create the Inner Query

In [6]:
sql_subquery1 = """SELECT t.team_long_name AS team,
                AVG(m.home_team_goal) AS home_avg
                FROM Match as m
                LEFT JOIN Team as t
                ON m.home_team_api_id = t.team_api_id
                WHERE season = '2011/2012'
                GROUP BY team;"""

In [7]:
# Run query
subquery1 = pd.read_sql(sql_subquery1, conn)
# Displaying data
subquery1.head(3)

,team,home_avg
0,1. FC Kaiserslautern,0.705882
1,1. FC Köln,1.176471
2,1. FC Nürnberg,1.294118


## Step 2: Place Subquery in the Main Query <code>FROM</code> Clause.

 - Add subquery in FROM Clause
 - Give subquery the alisas <code>AS subquery</code>
 - Add the ORDER BY Clause

In [8]:
sql_outter1 = """
                SELECT team, home_avg
                FROM (SELECT t.team_long_name AS team,
                        AVG(m.home_team_goal) AS home_avg
                        FROM Match as m
                        LEFT JOIN Team as t
                        ON m.home_team_api_id = t.team_api_id
                        WHERE season = '2011/2012'
                        GROUP BY team) AS subquery
                ORDER BY home_avg DESC
                LIMIT 3;
                """

## Step 3: Display Query Result

In [9]:
# Run query
view_data1 = pd.read_sql(sql_outter1, conn)
# Displaying data
view_data1

,team,home_avg
0,FC Barcelona,3.842105
1,Real Madrid CF,3.684211
2,PSV,3.352941


## <font color="blue">The Outcome</font> The top three home run scoring teams in the 2011-2012 season was Barcelona, Madris, and PSV.

### <font color="blue">Explanation:</font><br>

<strong>The INNER Subquery</strong><br>
<code>SELECT t.team_long_name AS team,
AVG(m.home_team_goal) AS home_avg
FROM Match as m
LEFT JOIN Team as t
ON m.home_team_api_id = t.team_api_id
WHERE season = '2011/2012'
GROUP BY team</code>

<br>
<br>
This subquery selects to columns <br>
 1. from matches table: home_team_goal
 2. from teams table: team_long_name
<br>
<br>
A mathematical calculation function was applied to one of the columns<br>
 1. <code>AVG(home_team_goal)</code>
<br>
<br>
The columns were then aliased<br>
 1. <code>t.team_long_name AS team</code><br>
 2. <code>AVG(m.home_team_goal) AS home_avg</code>
 <br>
 <br>
The subquery also conducts a left join between:<br>
 - <code>Match</code> LEFT JOIN <code>Team</code><br>
 - Aligns the data by <code>ON m.home_team_api_id = t.team_api_id</code><br>
<br>
The subquery then filteres by the <code>WHERE</code> clause, and returns all instances where the <code>season</code> contains '2011/2012'


<strong>The Outer Subquery in the FROM Clause</strong><br>

The subquery columns were selected<br>
<code>SELECT team, home_avg</code><br>
<br>
The outter query then specifies that these columns come <code>FROM</code>
the subquery.<br>

Since we want the top scorers, we then sorted the data by descending:<br>
<code> ORDER BY home_avg DESC</code><br>

The resuls were limited to the top 3 teams<br>
<code> LIMIT 3</code>

## What to remember

 - You can perform multiple subqueries in FROM statement
 - You can join a subquery to a table in FROM statement

# Example 2: Joining Subqueries in <code>FROM</code> clause

## <font color="red">Goal:</font><br>Generate a subquery using the match table, and then join that subquery to the country table to calculate information about matches with 10 or more goals in total

## Step 1: Create the Inner Subquery

In [10]:
sql_subquery2 = """SELECT m.country_id,
                m.id
                FROM Match as m
                WHERE (home_team_goal + away_team_goal) >= 10;"""

In [11]:
# Run query
subquery2 = pd.read_sql(sql_subquery2, conn)
# Displaying data
subquery2.head(5)

,country_id,id
0,1729,2157
1,1729,3093
2,1729,3369
3,1729,3566
4,4769,5192


### <font color="blue">Explanation:</font><br>

<strong>The INNER Subquery</strong><br>
<code>SELECT m.country_id,
m.id
FROM Match as m
WHERE (home_team_goal + away_team_goal) >= 10;</code><br>

<strong>The output</strong>
Has returned a dataframe that contains matches ('id') and countries ('country_id) that had more than 10 goals in that game.<br>

This is great BUT we need to get the names of the countries, not the IDs.<br>

To this end, we will need to combine the table from this query, with the country table, which has this information.<br>

<strong>Next steps:</strong><br>
1. The SELECT clause will have columns that describe the:<br>
 - Country name
 - Number of matches that had >10 goals
 
2. The FROM clause will refer to the country column<br>
<br>
3. We will need an inner join with the subquery<br>
<br>
4. Since this is a join, we need to specify which columns will be used to combine the main query (country) and the subquery table.<br>
<br>
5. Finally, we group the data by country, and order the results by number of matches with high scores in descending order.


## Step 2: Add Inner Query to Main Query

In [12]:
sql_outter2 = """
SELECT c.name AS country_name, COUNT(subquery.id) AS matches
FROM country AS c
INNER JOIN 
(SELECT country_id, id
        FROM Match
        WHERE (home_team_goal + away_team_goal) >= 10) AS subquery
ON c.id = subquery.country_id
GROUP BY country_name
ORDER BY matches DESC;
"""

## Step 3: Display Query Result

In [13]:
# Run query
mainquery2 = pd.read_sql(sql_outter2, conn)
# Displaying data
mainquery2.head(5)

,country_name,matches
0,Spain,5
1,England,4
2,Netherlands,2
3,Scotland,1
4,Germany,1


## <font color="blue">The Outcome</font> The countries with teams that have more than 10 goals in a match are Spain, England, Netherlands, Scotland, and Germany.

# Example 3: Getting More Granular with Joining Subqueries in the <code>FROM</code> clause

## <font color="red">Goal:</font><br> Using the previous example, create a table that also includes the scores made by the home team, and the away team.

## Step 1: Create the Inner Subquery

In [14]:
sql_subquery3 = """
SELECT c.name AS country,
m.date,
m.home_team_goal,
m.away_team_goal,
(m.home_team_goal + m.away_team_goal) AS total_goals
FROM Match as m
LEFT JOIN country as c
ON m.country_id = c.id;"""

In [15]:
# Run query
subquery3 = pd.read_sql(sql_subquery3, conn)
# Displaying data
subquery3.head(5)

,country,date,home_team_goal,away_team_goal,total_goals
0,Belgium,2008-08-17 00:00:00,1,1,2
1,Belgium,2008-08-16 00:00:00,0,0,0
2,Belgium,2008-08-16 00:00:00,0,3,3
3,Belgium,2008-08-17 00:00:00,5,0,5
4,Belgium,2008-08-16 00:00:00,1,3,4


### <font color="blue">Explanation:</font><br>

<strong>The INNER Subquery</strong><br>
<code>SELECT c.name AS country,
m.date,
m.home_team_goal,
m.away_team_goal,
(m.home_team_goal + m.away_team_goal) AS total_goals
FROM Match as m
LEFT JOIN country as c
ON m.country_id = c.id;</code><br>

<strong>The output</strong>
Has returned a dataframe that contains the country name, date, and goals made by the home team, the away team, and the total goals made.<br>

This is great but we wanted to filter the table for matches/games that had more than 10 games. If you see this table, it has all matches, even though that scores less than 3 goals.<br>

To this end, we will need to filter this subquery table in the main query.<br>

<strong>the table in the FROM clause is our subquery, so we will select columns from our subquery in the main clause!</strong>
<br>

<strong>Next steps:</strong><br>
1. The SELECT clause will have all the same columns from the subquery:<br>
 - Country name
 - Date of match
 - Home team goals scored
 - Away team goals scored
 - The total goals scored in that match
 
2. The FROM clause will instruct the RDMS to get these columns from the subquery table<br>
<br>
3. We will NOW filter the data for totals goals that have >= 10 using the <code>WHERE</code> clause<br>
<strong>DONE!</strong>

## Step 2: Add Inner Query to Main Query

In [16]:
sql_outter3 = """
SELECT country, date, home_team_goal, away_team_goal
FROM (SELECT c.name AS country,
    m.date,
    m.home_team_goal,
    m.away_team_goal,
    (m.home_team_goal + m.away_team_goal) AS total_goals
    FROM Match as m
    LEFT JOIN country as c
    ON m.country_id = c.id) AS subquery
WHERE total_goals >= 10;
"""

## Step 3: Display Query Result

In [17]:
# Run query
mainquery3 = pd.read_sql(sql_outter3, conn)
# Displaying data
mainquery3.head(5)

,country,date,home_team_goal,away_team_goal
0,England,2009-11-22 00:00:00,9,1
1,England,2011-08-28 00:00:00,8,2
2,England,2012-12-29 00:00:00,7,3
3,England,2013-05-19 00:00:00,5,5
4,France,2009-11-08 00:00:00,5,5
